In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import scipy as scp
from scipy import stats
import pingouin as pg
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import ipywidgets
from ipywidgets import interact, interact_manual
import os
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from ipyfilechooser import FileChooser
from src.utils.Evaluation import get_pvals_corrected, get_pvals_uncorrected

In [4]:
path_chooser = FileChooser(os.path.join(os.getcwd(),'/mnt/sds/sd20i001/sven/code/dynamic-cmr-models/exp/p2p/dmd/jcmr/iso2_5_32_p2p/img05_flow01_msk1/ssimssim_grad_dice/'), 'df_DMD_time_p2p.csv')
display(path_chooser)
@interact_manual
def get_segmental_correlations(alpha0='0.05', target=['lge', 'lgeef']):
    
    global msk_ss, df_pvals_corrected, df_pvals_uncorrected
    alpha0 = float(alpha0)
    strain = 'RS'
    phase_idx = 0
    phases = ['ED','MS','ES','PF','MD']
    
    
    path_to_our_dmd = path_chooser.selected
    df_DMD = pd.read_csv(path_to_our_dmd)
    df_pvals_uncorrected = get_pvals_uncorrected(df_DMD, target=target)
    print(df_pvals_uncorrected.shape)
    msk_ss, df_pvals_corrected = get_pvals_corrected(df_pvals_uncorrected, alpha0=alpha0)
    
    for c in df_pvals_corrected.columns:
        if c >=5:
            strain = 'CS'
            phase_idx = phase_idx%5
        sig_segments = df_pvals_corrected.index[df_pvals_corrected[c]<0.05].tolist()
        pvalues = df_pvals_corrected.iloc[sig_segments,c].tolist()
        if sig_segments: print('{} strain in phase: {} - significant AHA segments: {}, p-values: {}'.format(strain,phases[phase_idx], sig_segments, pvalues))
        phase_idx+=1

In [6]:
df_pvals_uncorrected